### Segmenting and Clustering Neighborhoods in Toronto

### PART 2 geographical coordinates for the neighbourhood

In [1]:
# importing required libraries
from bs4 import BeautifulSoup
import requests
import pandas as pd
print('libraries are imported')

libraries are imported


In [2]:
# gathering data from Wikipedia
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
source = requests.get(url).text
print('data is gathered')

data is gathered


In [3]:
soup= BeautifulSoup(source, 'lxml')

#### Transforming the data into a Pandas Dataframe

In [4]:
# defining the dataframe columns
column_names = ['PostalCode', 'Borough', 'Neighbourhood'] 

# instantiating the dataframe
neighbourhoods = pd.DataFrame(columns=column_names)

In [5]:
# loop to fill dataframe
table=soup.find('table')
for tr_table in table.find_all('tr'):
    raw_data=[]
    for td_table in tr_table.find_all('td'):
        raw_data.append(td_table.text.strip())
    if len(raw_data)==3:
        neighbourhoods.loc[len(neighbourhoods)] = raw_data


#### Data Cleansing

In [15]:
#removing borough cells that are ''not assigned'
neighbourhoods = neighbourhoods[neighbourhoods.Borough!='Not assigned']
neighbourhoods = neighbourhoods[neighbourhoods.Borough!= 0]
neighbourhoods.reset_index(drop = True, inplace = True)
i = 0
for i in range(0,neighbourhoods.shape[0]):
    if neighbourhoods.iloc[i][2] == 'Not assigned':
        neighbourhoods.iloc[i][2] = neighbourhoods.iloc[i][1]
        i = i+1
                                 
df = neighbourhoods.groupby(['PostalCode','Borough'])['Neighbourhood'].apply(', '.join).reset_index()



In [ ]:
df2=df[df.PostalCode != 'Not assigned']

In [16]:
print('Dataframe shape is', df2.shape)

Dataframe shape is (103, 3)


### Downloading Geo Coordinates Data

In [35]:
toronto_geo = 'https://cocl.us/Geospatial_data'
geo_data = pd.read_csv(toronto_geo)
geo_data.head()


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [36]:
#Changing column name into PostalCode for merging dfs.
geo_data_new = geo_data.copy()
geo_data_new.rename(columns={'Postal Code': 'PostalCode'}, inplace=True)


In [37]:
geo_data_new.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [38]:
# merging geo data and neighbourhood data
df_merged = pd.merge(geo_data_new, df2, on='PostalCode')

In [39]:
df_merged.head()

,PostalCode,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
